In [6]:
!pip install transformers sentencepiece accelerate bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.4 MB/s eta 0:00:00


In [19]:
import torch
from transformers import GenerationConfig, LlamaTokenizer, LlamaForCausalLM
import re

In [3]:
def generate_prompt(instruction: str, input_ctxt: str = None) -> str:
    if input_ctxt:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_ctxt}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [4]:
input_ctxt = '''
and its grammatical case is accusative. μονογενής is a adj, and its semantic 
domain information is 058007. The grammatical gender of this word is masculine, 
meaning “only begotten.” The lemma form of this word is μονογενής, 
The Louw-Nida semantic domain entry for this word is 58.52. and it is 
classified as a A-ASM and its grammatical number is singular. with a Strong’s 
number of 3439. This word functions within 3 discourse feature(s), including  
Focus+, which is defined as Constituents placed in P2 to give them focal 
prominence. Verb Focus+, which is defined as Verb in final position in clause 
demonstrates verb focus. Main clauses, which is defined as Main clauses are the 
top-level clauses in a sentence. 
They are the clauses that are not embedded in other clauses.
'''

In [7]:
tokenizer = LlamaTokenizer.from_pretrained("chainyo/alpaca-lora-7b")
model = LlamaForCausalLM.from_pretrained(
    "chainyo/alpaca-lora-7b",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
generation_config = GenerationConfig(
    temperature=0.2,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128,
)

model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --targ

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/39 [00:00<?, ?it/s]

In [9]:
prompt = generate_prompt('Generate a list of 5 questions based off of the input.', input_ctxt)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(model.device)

with torch.no_grad():
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
    )

In [16]:
questions = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print(questions)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a list of 5 questions based off of the input.

### Input:

and its grammatical case is accusative. μονογενής is a adj, and its semantic 
domain information is 058007. The grammatical gender of this word is masculine, 
meaning “only begotten.” The lemma form of this word is μονογενής, 
The Louw-Nida semantic domain entry for this word is 58.52. and it is 
classified as a A-ASM and its grammatical number is singular. with a Strong’s 
number of 3439. This word functions within 3 discourse feature(s), including  
Focus+, which is defined as Constituents placed in P2 to give them focal 
prominence. Verb Focus+, which is defined as Verb in final position in clause 
demonstrates verb focus. Main clauses, which is defined as Main clauses are the 
top-level clauses in a sentence. 
They are the clauses that are no

In [17]:
questions = questions.split('### Response:')[1]

In [26]:
questions

'\n1. What is the semantic domain information of μονογενής?\n2. What is the grammatical gender of μονογενής?\n3. What is the lemma form of μονογενής?\n4. What is the grammatical number of μονογενής?\n5. What discourse feature(s) does μονογενής function within?'

In [36]:
qs = re.sub('(\\n\d. )', 'split', questions)
qs = re.split('split', qs)
qs.pop(0)
qs

['What is the semantic domain information of μονογενής?',
 'What is the grammatical gender of μονογενής?',
 'What is the lemma form of μονογενής?',
 'What is the grammatical number of μονογενής?',
 'What discourse feature(s) does μονογενής function within?']

In [39]:
data = []

for question in qs:
  d = {'instruction': '', 'input': '', 'output': '', 'text': ''}

  prompt = generate_prompt(question, input_ctxt)
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  input_ids = input_ids.to(model.device)

  with torch.no_grad():
      outputs = model.generate(
          input_ids=input_ids,
          generation_config=generation_config,
          return_dict_in_generate=True,
          output_scores=True,
      )

  response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

  d['instruction'] = question
  d['input'] = input_ctxt
  d['output'] = response.split('### Response:')[1]
  d['text'] = response

  data.append(d)

data

[{'instruction': 'What is the semantic domain information of μονογενής?',
  'input': '\nand its grammatical case is accusative. μονογενής is a adj, and its semantic \ndomain information is 058007. The grammatical gender of this word is masculine, \nmeaning “only begotten.” The lemma form of this word is μονογενής, \nThe Louw-Nida semantic domain entry for this word is 58.52. and it is \nclassified as a A-ASM and its grammatical number is singular. with a Strong’s \nnumber of 3439. This word functions within 3 discourse feature(s), including  \nFocus+, which is defined as Constituents placed in P2 to give them focal \nprominence. Verb Focus+, which is defined as Verb in final position in clause \ndemonstrates verb focus. Main clauses, which is defined as Main clauses are the \ntop-level clauses in a sentence. \nThey are the clauses that are not embedded in other clauses.\n',
  'output': '\nThe semantic domain information of μονογενής is 058007.',
  'text': 'Below is an instruction that 